In [1]:
import numpy as np
import pandas as pd
pd.pandas.set_option('display.max_columns',None)

import seaborn as sns
sns.set(font_scale=1.2)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss

from sklearn.metrics import confusion_matrix

import pickle
import re

from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

import docx
from docx.shared import Inches
import datetime
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
df = pd.read_csv(r'part6\Historical Weather Data 2010-2021_preprocessed_1.csv')
df.head()

,observation,date,month,year,tempC_7to8,tempC_1to2,tempC_6to7,tempC_avg(0C),Relative humidity_7to8,Relative humidity_1to2,Relative humidity_6to7,Relative humidity_avg(%),windspeedKmph_7to8,windspeedKmph_1to2,windspeedKmph_6to7,windspeedKmph_avg(Km/h),pressureMB_7to8,pressureMB_1to2,pressureMB_6to7,pressureMB_avg,precipMM_7to8,precipMM_1to2,precipMM_6to7,precipMM_avg(mm),weatherDesc_7to8,weatherDesc_1to2,weatherDesc_6to7,weatherDesc,Sunshine Hours,%_soil_moisure,soil_pH,water_pH,water_TDS_mgpl,Label (Disease Yes/No),Type of Disease (Bacterial Blight/Telya),Anthracnose,Fruit Spot/ Rot,Fusarium Wilt,Fruit Borer / Blight Blora
0,2010-01-01,1,1,2010,20,30,20,23,42,33,59,44,9,2,4,5,1015,1012,1013,1013,0.0,0.0,0.0,0.0,5,5,5,5,9.8,45,6.91,7.18,1709.0,0,0,0,0,0,0
1,2010-01-02,2,1,2010,23,29,23,25,49,40,62,50,9,3,3,5,1015,1013,1015,1014,0.0,0.0,0.0,0.0,5,4,4,4,9.8,45,6.77,7.66,1707.0,1,1,1,0,0,0
2,2010-01-03,3,1,2010,24,27,21,24,61,50,78,63,4,8,6,6,1017,1014,1015,1015,0.0,0.0,0.0,0.0,4,4,4,4,9.8,41,6.76,7.44,1707.0,0,0,0,0,0,0
3,2010-01-04,4,1,2010,23,29,20,24,57,30,52,46,5,7,6,6,1016,1012,1013,1013,0.0,0.0,0.0,0.0,5,5,5,5,9.8,43,7.10,7.43,1708.0,0,0,0,0,0,0
4,2010-01-05,5,1,2010,22,30,21,24,48,34,54,45,6,2,4,4,1013,1010,1012,1011,0.0,0.0,0.0,0.0,5,5,5,5,9.8,44,6.53,8.05,1709.0,0,0,0,0,0,0


In [3]:
# columns name 
print(list(df.columns))

['observation', 'date', 'month', 'year', 'tempC_7to8', 'tempC_1to2', 'tempC_6to7', 'tempC_avg(0C)', 'Relative humidity_7to8', 'Relative humidity_1to2', 'Relative humidity_6to7', 'Relative humidity_avg(%)', 'windspeedKmph_7to8', 'windspeedKmph_1to2', 'windspeedKmph_6to7', 'windspeedKmph_avg(Km/h)', 'pressureMB_7to8', 'pressureMB_1to2', 'pressureMB_6to7', 'pressureMB_avg', 'precipMM_7to8', 'precipMM_1to2', 'precipMM_6to7', 'precipMM_avg(mm)', 'weatherDesc_7to8', 'weatherDesc_1to2', 'weatherDesc_6to7', 'weatherDesc', 'Sunshine Hours', '%_soil_moisure', 'soil_pH', 'water_pH', 'water_TDS_mgpl', 'Label (Disease Yes/No)', 'Type of Disease (Bacterial Blight/Telya)', 'Anthracnose', 'Fruit Spot/ Rot', 'Fusarium Wilt', 'Fruit Borer / Blight Blora']


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4227 entries, 0 to 4226
Data columns (total 39 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   observation                               4227 non-null   object 
 1   date                                      4227 non-null   int64  
 2   month                                     4227 non-null   int64  
 3   year                                      4227 non-null   int64  
 4   tempC_7to8                                4227 non-null   int64  
 5   tempC_1to2                                4227 non-null   int64  
 6   tempC_6to7                                4227 non-null   int64  
 7   tempC_avg(0C)                             4227 non-null   int64  
 8   Relative humidity_7to8                    4227 non-null   int64  
 9   Relative humidity_1to2                    4227 non-null   int64  
 10  Relative humidity_6to7              

In [5]:
lr_Classifier = LogisticRegression(max_iter=1000, random_state=3757)  
knn_Classifier = KNeighborsClassifier(n_jobs=-1, n_neighbors=5, p=2, weights='uniform')  
nb_Classifier = GaussianNB(priors=None, var_smoothing=1e-09)
svm_Classifier = SVC(gamma='auto')
dt_Classifier = DecisionTreeClassifier(random_state=3757)
rf_Classifier = RandomForestClassifier(n_jobs=-1, random_state=3757)
gb_Classifier = GradientBoostingClassifier(random_state=3757)
ab_Classifier = AdaBoostClassifier(random_state=3757)

classifiers = [lr_Classifier, knn_Classifier, nb_Classifier, svm_Classifier, dt_Classifier, 
               rf_Classifier, gb_Classifier, ab_Classifier]

model_name = ['lr_Classifier', 'knn_Classifier', 'nb_Classifier', 'svm_Classifier', 'dt_Classifier', 
              'rf_Classifier', 'gb_Classifier', 'ab_Classifier']

In [6]:
X = df[['tempC_avg(0C)', 'Relative humidity_avg(%)', 'windspeedKmph_avg(Km/h)', 'pressureMB_avg', 
        'precipMM_avg(mm)', 'weatherDesc', 'Sunshine Hours', '%_soil_moisure']]
Y = df[['Type of Disease (Bacterial Blight/Telya)', 'Anthracnose', 'Fruit Spot/ Rot', 
        'Fusarium Wilt', 'Fruit Borer / Blight Blora']]

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)

In [7]:
X_train.head(3)

,tempC_avg(0C),Relative humidity_avg(%),windspeedKmph_avg(Km/h),pressureMB_avg,precipMM_avg(mm),weatherDesc,Sunshine Hours,%_soil_moisure
528,25,83,21,1001,0.166667,3,6.2,52
176,24,91,6,1004,0.100000,3,3.8,65
422,28,15,7,1010,0.000000,5,10.7,35


In [8]:
y_train.head(3)

,Type of Disease (Bacterial Blight/Telya),Anthracnose,Fruit Spot/ Rot,Fusarium Wilt,Fruit Borer / Blight Blora
528,1,1,0,0,1
176,0,0,1,0,1
422,0,0,0,0,0


In [9]:
def builMultiLabelModels(X_train, X_test, y_train, y_test, classifierModel):
    
    multiLabelModel = MultiOutputClassifier(classifierModel, n_jobs=2)
    
    classifierModel = multiLabelModel.fit(X_train, y_train)
    
    y_pred = classifierModel.predict(X_test)
    
    print("Accuracy_score:", round((accuracy_score(y_test, y_pred))*100,2),'%')
    print("Loss:", round((1-accuracy_score(y_test, y_pred))*100,2),'%')
    print("Hamming_loss:", round((hamming_loss(y_test, y_pred))*100,2),'%')
    print("Classification_report:\n",metrics.classification_report(y_test, y_pred))
    
    return classifierModel, {'Accuracy': round((accuracy_score(y_test, y_pred))*100,2), 
                             'Loss': round((1-accuracy_score(y_test, y_pred))*100,2), 
                             'Hamming_loss': round((hamming_loss(y_test, y_pred))*100,2)
                            }

In [10]:
def get_MultiLabelModelResult(classifiers, model_name):
    
    
    fname_date = datetime.datetime.now()  
    
    if not os.path.exists("images"): os.mkdir("images")
    if not os.path.exists("docs"): os.mkdir("docs")

    result_df = pd.DataFrame()

    for cls in range(len(classifiers)):
        print(f'Model : {model_name[cls]}')

        model, data = builMultiLabelModels(X_train, X_test, y_train, y_test, classifiers[cls])

        # save the model to disk
        filename = f'MultiLabelModel_{model_name[cls]}'
        filename = re.sub('\W+','_', filename )+'.pkl'
        pickle.dump(model, open(filename, 'wb'))
        print(f'File saved : {filename}')

        data['Model_name'] = model_name[cls]
        data['Filename'] = filename

        print(f'data : {data}')

        result_df = result_df.append(data, ignore_index=True)

    fname_date = ''.join(e for e in str(fname_date) if e.isalnum())

    
    result_df1 = result_df[['Model_name', 'Accuracy', 'Loss', 'Hamming_loss', 'Filename']]
    result_df1.to_csv(f'part6//MultiLabelModel_result_{fname_date}.csv', index=False)
    
    return result_df1


In [11]:
def new_ensemble_voting_classifier():
    
#     selecting top 3 models
    models_df = get_MultiLabelModelResult(classifiers, model_name)
    models_list = list(models_df.sort_values(by = ['Accuracy'], ascending = False)[0:3]['Filename'].values)
    
#     get model prediction results
    output_list = []
    for path in models_list:
        model = pickle.load(open(path, 'rb')) 
        pred = model.predict(X_test)
        output_list.append(pred)

#     get predicted values (from votting)
    and_of_pred = (output_list[0] & output_list[1] & output_list[2])
    or_of_pred = (output_list[0] | output_list[1] | output_list[2])
    y_pred_new = np.nan_to_num(and_of_pred // or_of_pred)
    
    print("Accuracy_score:", round((accuracy_score(y_test, y_pred_new))*100,2),'%')
    print("Loss:", round((1-accuracy_score(y_test, y_pred_new))*100,2),'%')
    print("Hamming_loss:", round((hamming_loss(y_test, y_pred_new))*100,2),'%')
    

In [12]:
new_ensemble_voting_classifier()

Model : lr_Classifier
Accuracy_score: 72.42 %
Loss: 27.58 %
Hamming_loss: 7.38 %
Classification_report:
               precision    recall  f1-score   support

           0       0.72      0.60      0.66       319
           1       0.39      0.18      0.24       164
           2       0.89      0.91      0.90       162
           3       1.00      0.95      0.97        38
           4       0.87      0.92      0.90       243

   micro avg       0.79      0.68      0.73       926
   macro avg       0.78      0.71      0.73       926
weighted avg       0.74      0.68      0.70       926
 samples avg       0.29      0.28      0.27       926

File saved : MultiLabelModel_lr_Classifier.pkl
data : {'Accuracy': 72.42, 'Loss': 27.58, 'Hamming_loss': 7.38, 'Model_name': 'lr_Classifier', 'Filename': 'MultiLabelModel_lr_Classifier.pkl'}


c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model : knn_Classifier
Accuracy_score: 84.24 %
Loss: 15.76 %
Hamming_loss: 3.96 %
Classification_report:
               precision    recall  f1-score   support

           0       0.81      0.71      0.76       319
           1       0.79      0.90      0.84       164
           2       0.95      0.97      0.96       162
           3       0.89      0.66      0.76        38
           4       0.93      0.98      0.96       243

   micro avg       0.87      0.86      0.86       926
   macro avg       0.88      0.84      0.85       926
weighted avg       0.87      0.86      0.86       926
 samples avg       0.34      0.35      0.34       926

File saved : MultiLabelModel_knn_Classifier.pkl
data : {'Accuracy': 84.24, 'Loss': 15.76, 'Hamming_loss': 3.96, 'Model_name': 'knn_Classifier', 'Filename': 'MultiLabelModel_knn_Classifier.pkl'}
Model : nb_Classifier
Accuracy_score: 62.57 %
Loss: 37.43 %
Hamming_loss: 14.56 %
Classification_report:
               precision    recall  f1-score   suppo

c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

Accuracy_score: 78.57 %
Loss: 21.43 %
Hamming_loss: 5.99 %
Classification_report:
               precision    recall  f1-score   support

           0       0.86      0.59      0.70       319
           1       0.84      0.61      0.71       164
           2       0.99      0.72      0.83       162
           3       1.00      0.03      0.05        38
           4       0.97      0.82      0.89       243

   micro avg       0.91      0.65      0.76       926
   macro avg       0.93      0.55      0.63       926
weighted avg       0.91      0.65      0.75       926
 samples avg       0.28      0.27      0.27       926

File saved : MultiLabelModel_svm_Classifier.pkl
data : {'Accuracy': 78.57, 'Loss': 21.43, 'Hamming_loss': 5.99, 'Model_name': 'svm_Classifier', 'Filename': 'MultiLabelModel_svm_Classifier.pkl'}
Model : dt_Classifier
Accuracy_score: 92.91 %
Loss: 7.09 %
Hamming_loss: 1.58 %
Classification_report:
               precision    recall  f1-score   support

           0       0.

...

c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy_score: 95.35 %
Loss: 4.65 %
Hamming_loss: 1.06 %
Classification_report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.95       319
           1       0.87      0.97      0.92       164
           2       0.98      1.00      0.99       162
           3       1.00      0.95      0.97        38
           4       1.00      1.00      1.00       243

   micro avg       0.96      0.97      0.96       926
   macro avg       0.96      0.97      0.97       926
weighted avg       0.96      0.97      0.96       926
 samples avg       0.40      0.41      0.40       926

File saved : MultiLabelModel_ab_Classifier.pkl
data : {'Accuracy': 95.35, 'Loss': 4.65, 'Hamming_loss': 1.06, 'Model_name': 'ab_Classifier', 'Filename': 'MultiLabelModel_ab_Classifier.pkl'}


c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy_score: 95.67 %
Loss: 4.33 %
Hamming_loss: 0.93 %


c:\users\munge\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in floor_divide


In [13]:
# https://machinelearningmastery.com/voting-ensembles-with-python/
# Hard Voting. Predict the class with the largest sum of votes from models
# Soft Voting. Predict the class with the largest summed probability from models.
